In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
%matplotlib inline
import pandas as pd

/Users/bumho/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import json
from pprint import pprint

def getGameJSON(path):
    with open(path) as data_file:    
        data = json.load(data_file)
    return data
        
data = getGameJSON('GSWvsCLE.json')


In [3]:
# read in the court png file
court = plt.imread("fullcourt.png")

headers = ["team_id", "player_id", "x_loc", "y_loc", 
           "radius"]

def playerDFtoList(frame):
    xList = frame['x_loc'].tolist()
    yList = frame['y_loc'].tolist()
    bothList = []
    for i in range(len(xList)):
        bothList.append([ xList[i], yList[i] ])
    return np.array(bothList)

In [4]:
def getMoments(data):
    player_moments = []
    for event in data['events']:
        for moment in event['moments']:
            for player in moment[5]:
                player_moments.append(player)

    df = pd.DataFrame(player_moments, columns=headers)
    return df


In [11]:
min_5 = 25*60*6*11

def dist(data, ball):
    data["distanceToBall"] = np.sqrt((data["x_loc"] -ball["x_loc"])**2+ (data["y_loc"] -ball["y_loc"])**2)

def hasBall(data):
    data["hasBall"] = data["distanceToBall"].apply(lambda x: 1 if x < 2 else 0)
    
def transform(l, ball):
    ball.reset_index(drop=True, inplace =True)    
    result = []
    for player in l:
        player.reset_index(drop=True, inplace =True)
        dist(player, ball)
        hasBall(player)
        result.append(player[(player.hasBall == 1)][["player_id", "team_id", "x_loc", "y_loc"]])
    
    return pd.concat(result)

def removeRepeats(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]        

In [13]:
# Get moments from the zeroth event
player_moments = []
endsOfEvents = []
counter = -1
for event in data['events']:
    for moment in event['moments']:
        counter += 1
        for player in moment[5]:
            player_moments.append(player)
    endsOfEvents.append(counter)

df = pd.DataFrame(player_moments, columns=headers)
df = df.head(min_5)
endsOfEvents = removeRepeats(endsOfEvents)

In [14]:
# Get specific player's movements
ball = df[df.player_id==-1]

steph = df[df.player_id==201939]
klay = df[df.player_id==202691]
bogut = df[df.player_id==101106]
green = df[df.player_id==203110]
barnes = df[df.player_id==203084]

lebron = df[df.player_id==2544]
smith = df[df.player_id==2747]
love = df[df.player_id==201567]
irving = df[df.player_id==202681]
mozgov = df[df.player_id==202389]

# convert them to numpy arrays
ballA = playerDFtoList(ball)

stephA = playerDFtoList(steph)
klayA = playerDFtoList(klay)
bogutA = playerDFtoList(bogut)
greenA = playerDFtoList(green)
barnesA = playerDFtoList(barnes)

lebronA = playerDFtoList(lebron)
smithA = playerDFtoList(smith)
loveA = playerDFtoList(love)
irvingA = playerDFtoList(irving)
mozgovA = playerDFtoList(mozgov)

#Get ball radiuses
radii = ball['radius'].tolist()

In [15]:
l = [steph,klay,bogut ,green ,barnes ,lebron,smith,love,irving,mozgov]

a = transform(l, ball)

/Users/bumho/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bumho/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
a = a.sort_index()
idx = np.unique(a.index, return_index=True)[1]
a= a.iloc[idx]
index = np.arange(0, min_5/11, 1)

a = a.reindex(index, fill_value=0)
a_pos = playerDFtoList(a)

In [17]:
# Get the indexes that overlap
skipRanges = []
# Only get the first 17 indices b/c we're only doing the first 5 minutes of the game
for eventEndIndex in endsOfEvents[:18]:
    lastLocation = ballA[eventEndIndex]
#     print '       Event end at '+str(eventEndIndex)+':',lastLocation
    for i in range(eventEndIndex+1,len(ballA[eventEndIndex+1:])):
        if (ballA[i] == lastLocation).all():
#             print 'matched location at '+str(i)+':',ballA[i]
            skipRanges.append([eventEndIndex, i])
            break
print 'Ranges to skip:',skipRanges

Ranges to skip: [[149, 299], [922, 1360], [1360, 1510], [1885, 2410], [2410, 2560], [2885, 3360], [3360, 3510], [3881, 4031]]


In [10]:
def distance(ball):
    basket1 = [5.32, 24.8]
    basket2 = [88.55, 24.8]
    halfcourt = 47.0
    
    ball['cavsHoop'] = np.sqrt((basket1[0] - ball['x_loc'])**2 + (basket1[1] - ball['y_loc'])**2)
    ball['warriorsHoop'] = np.sqrt((basket2[0] - ball['x_loc'])**2 + (basket2[1] - ball['y_loc'])**2)
    
distance(ball)
a

/Users/bumho/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bumho/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,player_id,team_id,x_loc,y_loc
0,202681,1610612739,27.87412,45.48980
1,202681,1610612739,27.79347,45.43106
2,202681,1610612739,27.71793,45.36438
3,202681,1610612739,27.61261,45.33913
4,202681,1610612739,27.51496,45.30448
5,202681,1610612739,27.42254,45.26554
6,202681,1610612739,27.33418,45.22266
7,202681,1610612739,27.25215,45.17668
8,202681,1610612739,27.13954,45.15537
9,202681,1610612739,27.03458,45.13076


In [18]:
ex = {}
ex["Ball"] = ballA.tolist()

ex["Curry"] = stephA.tolist()
ex["Thompson"] = klayA.tolist()
ex["Bogut"] = bogutA.tolist()
ex["Green"] = greenA.tolist()
ex["Barnes"] = barnesA.tolist()


ex["James"] = lebronA.tolist()
ex["Smith"] = smithA.tolist()
ex["Love"] = loveA.tolist()
ex["Irving"] = irvingA.tolist()
ex["Mozgov"] = mozgovA.tolist()

ex["pos"] = a_pos.tolist()

# print ex['steph'][148:152]
# print ex['steph'][299:301]

# Rebuild the dictionary with skip ranges
for player in ex:
    # Loop through the skipRanges, going in reverse so we don't mess up indices
    for thisRange in reversed(skipRanges):
        # Delete the indices to skip
        del ex[player][thisRange[0]:thisRange[1]]

In [19]:
with open("./website/public/dump.json", "w") as outfile:
    json.dump(ex, outfile)